In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
# 위의 코드가 작동하지 않을 때 대안 코드 1 실행
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [1]:
from konlpy.tag import Mecab
mecab = Mecab()
#NameError: name 'Tagger' is not defined 오류 발생 시 런타임을 재실행 해주세요.

In [3]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [4]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1031/1031), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 3998 (delta 919), reused 851 (delta 851), pack-reused 2967 (from 2)
Receiving objects: 100% (3998/3998), 8.30 MiB | 17.41 MiB/s, done.
Resolving deltas: 100% (2529/2529), done.
/content/fastText
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++17 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++17 

In [5]:
#hgtk 설치
!pip install hgtk

## 1. 데이터 로드

In [2]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk

In [3]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x78151c31f3d0>)

In [4]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [5]:
total_data.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


## 2. hgtk 튜토리얼



word embedding이 단어 단위의 임베딩이었다면, character embedding은 문자 단위의 임베딩입니다. 한국어를 character embedding할 수 있는 것이 바로 자음 모음 분리기 hgtk입니다.

 영어는 하나의 알파벳(52자)를 기준으로 character embedding을 하지만, 한국어에서 하나의 음절별로 character embedding을 하면 11172개의 음절이 있기 때문에 계산량이 너무 많습니다. 따라서 그보다 작은 단위인 자음,모음으로 분리하는 것입니다.

 >참고 repo: https://github.com/bluedisk/hangul-toolkit

In [6]:
# 한글인지 체크
print(hgtk.checker.is_hangul('ㄱ'))
print(hgtk.checker.is_hangul('12'))
print(hgtk.checker.is_hangul('a'))

True
False
False


In [7]:
# 음절을 초성, 중성, 종성으로 분해
print(hgtk.letter.decompose('남'))
# 초성, 중성, 종성을 하나의 음절로 결합
print(hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ'))

('ㄴ', 'ㅏ', 'ㅁ')
남


In [8]:
# 결합할 수 없는 상황에서는 에러 발생
try:
  hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ') # 중성이 없는 경우
except:
  print('에러 발생')

에러 발생


## 3. 데이터 전처리

![image.png](attachment:image.png)

In [9]:
def word_to_jamo(token):
  def to_special_token(jamo): # 경우에 따라 초, 중, 종성이 다 있는 게 아닌 경우도 있다. 이 경우 -를 반환해주는 함수
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [10]:
print(word_to_jamo('남동생'))
print(word_to_jamo('야구')) # 야구의 경우 종성이 없으므로 종성 부분을 -로 반환

ㄴㅏㅁㄷㅗㅇㅅㅐㅇ
ㅇㅑ-ㄱㅜ-


In [11]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [12]:
# mecab으로 형태소를 분리해주고 그 형태소마다 각각 자음모음을 분리해주는 함수
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [13]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [14]:
# 리뷰 데이터의 reviews 컬럼만을 가져와서 자모 분리
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_numpy()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [00:57<00:00, 3457.57it/s]


In [15]:
print(len(tokenized_data))
print("전처리 전:", total_data['reviews'][1])
print("전처리 후:", tokenized_data[1])

200000
전처리 전: 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
전처리 후: ['ㅌㅐㄱㅂㅐ-', 'ㄱㅏ-', 'ㅇㅓㅇㅁㅏㅇ', 'ㅇㅣ-', 'ㄴㅔ-', 'ㅇㅛㅇ', 'ㅈㅓ-ㅎㅢ-', 'ㅈㅣㅂ', 'ㅁㅣㅌ', 'ㅇㅔ-', 'ㅊㅡㅇ', 'ㅇㅔ-', 'ㅁㅏㄹ', 'ㄷㅗ-', 'ㅇㅓㅄㅇㅣ-', 'ㄴㅘ-ㄷㅜ-', 'ㄱㅗ-', 'ㄱㅏ-', 'ㄱㅗ-']


단어를 자모 분리한 것을 역으로 하여 자모 상태를 단어로 다시 결합시키는 함수도 정의해봅시다. 이는 단어의 코사인 유사도를 평가할 때 자모 분리가 된 상태가 아니라 단어 상태로 편리하게 보기 위함입니다.

In [16]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

## 4. FastText

In [17]:
import fasttext

fasttext를 실행하기에 앞서 훈련 대상인 단어들을 txt 파일로 준비해둬야 합니다. 따라서 `tokenized_data.txt`라는 파일을 쓰기 모드(w)로 생성해주고 앞서 전처리한 `tokenized_data`를 입력해줍니다.

In [18]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 364660.25 line/s]


아래처럼 `train_unsupeviesd` 함수는 훈련을 시켜주는 함수입니다. 인자로 훈련할 단어가 담긴 txt 파일을 지정하고 model을 `cbow`나 `skipgram` 중에 하나를 고르면 됩니다.

In [19]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [20]:
model.save_model("fasttext.bin")

In [21]:
model = fasttext.load_model("fasttext.bin")

In [22]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([ 0.10053975,  0.5483576 ,  0.9365663 , -0.6376111 , -0.77265793,
       -0.4806171 ,  0.12574688,  0.77823585, -0.5412491 , -0.37715617,
        0.85062027,  0.27477103, -0.2019856 ,  0.34986034, -0.3151048 ,
        0.804251  , -0.18566827,  0.6056214 , -0.24055727,  0.37627244,
        0.6698768 , -0.5237914 ,  0.7521822 ,  0.02341442, -0.17149611,
       -0.5461813 ,  0.35431802,  1.0537186 ,  1.0209532 , -0.791429  ,
        0.68040353, -0.5201949 ,  0.2005935 , -0.5775247 ,  1.5114708 ,
       -0.2786645 , -0.3823303 , -0.23561004, -0.3335364 ,  0.12960154,
        0.38627267, -0.26394966,  0.18493988,  0.18794619, -0.5911801 ,
       -0.5651976 ,  0.17870744,  0.07417093,  0.6941467 , -0.35705242,
       -0.3284669 ,  0.09078441,  0.5500784 ,  0.19030026, -1.2988886 ,
        0.13273665,  0.27076134,  0.52283436, -0.65024686, -0.07252873,
       -0.06780522, -0.7910158 , -0.14931817,  0.7369413 ,  0.11122435,
        0.07620656, -0.14498606,  0.05974256, -0.25741562, -0.19

`get_nearest_neighbors` 함수를 사용하여 '남동생'이라는 단어와 가장 유사도가 높은 단어들(자모 분리된 상태)을 k개만큼 출력해줍니다.

In [23]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.885690450668335, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.830272376537323, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7768639326095581, 'ㅊㅣㄴㄱㅜ-'),
 (0.7601721882820129, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7577266097068787, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7378281354904175, 'ㅈㅗ-ㅋㅏ-'),
 (0.7079845070838928, 'ㄴㅏㅁㅇㅏ-'),
 (0.6951063871383667, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.692351758480072, 'ㅅㅓㄴㅁㅜㄹ'),
 (0.6873787641525269, 'ㅇㅓㄴㄴㅣ-')]

앞서 만든 `jamo_to_word`로 가독성이 좋게 출력해봅시다.

In [24]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [25]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))

[('동생', 0.885690450668335), ('남친', 0.830272376537323), ('친구', 0.7768639326095581), ('생일', 0.7601721882820129), ('남편', 0.7577266097068787), ('조카', 0.7378281354904175), ('남아', 0.7079845070838928), ('중학생', 0.6951063871383667), ('선물', 0.692351758480072), ('언니', 0.6873787641525269)]
[('구매처', 0.8597714304924011), ('구입', 0.8207437992095947), ('주문', 0.759748637676239), ('주문건', 0.7139223217964172), ('주문서', 0.6493059992790222), ('구매자', 0.6275764107704163), ('헤매', 0.5957227349281311), ('구명조끼', 0.5918538570404053), ('구이', 0.5916499495506287), ('수입', 0.5890144109725952)]
[('배송지', 0.818331241607666), ('깨달', 0.7824680805206299), ('매달', 0.7608924508094788), ('택배', 0.7438911199569702), ('배소', 0.7410486936569214), ('직송', 0.7350515127182007), ('운송장', 0.7219671607017517), ('메달', 0.7188259959220886), ('송장', 0.710339367389679), ('주소', 0.7091870903968811)]


## 5. Word2Vec

![image.png](attachment:image.png)
이제 word2vec를 사용하여 자모 단위로 분리하는 것이 아닌 단어 단위로 분리하여 임베딩 벡터를 생성해볼 것입니다.

In [26]:
# 간단하게 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:22<00:00, 8752.62it/s]


In [27]:
print("word2vec용 데이터:", tokenized_data2[0])
print("fasttext용 데이터:", tokenized_data[0])

word2vec용 데이터: ['배공', '빠르', '고', '굿']
fasttext용 데이터: ['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [ ]:
!pip install gensim

In [28]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [29]:
# 완성된 임베딩 매트릭스의 크기 확인
# 단어의 총 개수는 15005개이고 벡터 차원은 1000으로 축소되었다.
model2.wv.vectors.shape

(15005, 1000)

## 6. FastText와 Word2Vec 결과 비교

### **Q1) 남동생과 주문 두 단어를 input으로 넣고 결과를 비교한 뒤 해석해보세요**

*(Hint: 유사도, 단어의 의미, 단어의 생김새 등을 고려해 볼 수 있습니다)*

In [30]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("남동생"))

FastText 유사도: [('동생', 0.885690450668335), ('남친', 0.830272376537323), ('친구', 0.7768639326095581), ('생일', 0.7601721882820129), ('남편', 0.7577266097068787), ('조카', 0.7378281354904175), ('남아', 0.7079845070838928), ('중학생', 0.6951063871383667), ('선물', 0.692351758480072), ('언니', 0.6873787641525269)]
Word2Vec 유사도: [('양가', 0.7420191764831543), ('사촌', 0.7381390333175659), ('오빠', 0.7253308296203613), ('졸업', 0.718350350856781), ('입학', 0.7167815566062927), ('앞둔', 0.7045868635177612), ('친한', 0.7030236124992371), ('개업', 0.7015496492385864), ('시어머님', 0.698081910610199), ('기념일', 0.697049617767334)]


In [31]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('주문'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("주문"))

FastText 유사도: [('주문건', 0.9069121479988098), ('주문서', 0.8590660691261292), ('주문자', 0.7641658782958984), ('구매', 0.7597488164901733), ('구입', 0.7431609630584717), ('주무시', 0.7306575179100037), ('구매처', 0.7268259525299072), ('주무', 0.6992968916893005), ('시켰었', 0.6729649305343628), ('시킨', 0.6665186285972595)]
Word2Vec 유사도: [('구입', 0.8177275657653809), ('구매', 0.815786600112915), ('선택', 0.6366980075836182), ('시켰', 0.5657947063446045), ('장만', 0.562260627746582), ('도전', 0.5495457053184509), ('결제', 0.5471669435501099), ('준비', 0.5102933049201965), ('시킨', 0.5046722292900085), ('신청', 0.5021069645881653)]


**Your Inference:**

FastText
- 음절, 자모(subword) 단위까지 임베딩 학습
- 남동생 -> 동생, ‘남친’처럼 공통 자모(남, 동)가 포함된 단어가 높은 점수
- 주문 -> 주문건, 주문서처럼 어근 + 파생어도 쉽게 포착
- 자모 중복으로 의미적으로 무관한 주무시, 주무(주문과 철자 일부만 겹침) 같은 노이즈도 등장

Word2Vec
- 단어 단위(one-hot 토큰) + 맥락 기반
- 남동생과 같이 쓰이는 상황, 관계(사촌, 오빠, 졸업/입학 행사) 중심 추천
- 주문과 동일 행위(구매, 결제)나 행위 과정(준비, 선택)이 상위
- 형태가 비슷하다고 해서 자동으로 높아지지는 않음

FastText는 형태적 근접성이 주도, Word2Vec은 의미적, 상황적 근접성이 주도한다.


### **Q2) Fasttext가 Word2Vec보다 항상 성능이 나은지 다양한 input을 넣어서 시도해보세요**

In [33]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('소고기'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("소고기"))

FastText 유사도: [('콩고기', 0.9378204941749573), ('닭고기', 0.9362733364105225), ('소불고기', 0.9294338226318359), ('양고기', 0.9142380952835083), ('고기', 0.9004213809967041), ('돼지고기', 0.8941742777824402), ('맛보기', 0.8826991319656372), ('물고기', 0.8702911734580994), ('소갈비', 0.8608219623565674), ('살코기', 0.854370653629303)]
Word2Vec 유사도: [('장어', 0.7731940150260925), ('치즈', 0.754091739654541), ('샐러드', 0.7534346580505371), ('탕', 0.7517513632774353), ('구이', 0.7462443113327026), ('꿀', 0.7428021430969238), ('총각김치', 0.7400710582733154), ('치킨', 0.739612877368927), ('양념장', 0.7362431883811951), ('즙', 0.7271473407745361)]


In [39]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('간단'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("간단"))

FastText 유사도: [('간당간당', 0.8823781609535217), ('간단히', 0.8742222785949707), ('간편', 0.8486782312393188), ('간편히', 0.7952190041542053), ('간섭', 0.7399829030036926), ('간접', 0.7379177212715149), ('편리', 0.7355092167854309), ('단단', 0.7162139415740967), ('간다', 0.709976851940155), ('판단', 0.6764598488807678)]
Word2Vec 유사도: [('간편', 0.8912436962127686), ('수월', 0.7677561044692993), ('편리', 0.7585481405258179), ('용이', 0.7114496231079102), ('쉽', 0.685396134853363), ('깔끔', 0.6480114459991455), ('쉬웠', 0.6448359489440918), ('말끔', 0.6338902711868286), ('조용', 0.6288580894470215), ('쉬워서', 0.6253777742385864)]


In [40]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('배달앱'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("배달앱"))

FastText 유사도: [('배달', 0.9379811882972717), ('배송지', 0.8406723737716675), ('배소', 0.7974721789360046), ('배공', 0.7883846163749695), ('백배', 0.7591103911399841), ('택배', 0.7412457466125488), ('배상', 0.7188317775726318), ('배관', 0.7172881960868835), ('배경', 0.7132687568664551), ('배송', 0.7094148397445679)]


KeyError: "Key '배달앱' not present in vocabulary"

**Your Inference:**

- 형태(스펠링) 유사 단어 추천이 필요할 때 FastText가 유리함. 맞춤법 교정, 신조어 및 오타 대응, 어근 활용형 확장 등에 강점.
- 맥락적·의미적 유사 단어 추천이 필요할 때 Word2Vec이 유리함. 동의어 제안, 문장과 문서 의미 비교, 추천 시스템 등에서 실수가 적음.
- 노이즈 관리
  - FastText의 subword 특성 때문에 하위 문자를 공유하나 의미가 다른 단어가 섞일 수 있음.
  - Word2Vec은 희귀 단어 처리가 불안정(등장 빈도가 낮으면 벡터 품질 떨어짐)하니 학습 말뭉치 규모나 도메인 적합성을 확인해야함.
  - ex) 작은 전용 코퍼스로 도메인 특화 임베딩시 FastText를 사용하면 전문 용어 많아도 OOV 적음.
